In [2]:
import time

import pandas as pd
import numpy as np
import random
from tqdm import tqdm

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# 自作関数
from dataload.data_load import data_load
from func.my_func import *

In [6]:
INPUT_DIR = '../data/'

df = pd.read_csv(INPUT_DIR + '1504_player_df.csv')
X = df.drop([
    'TargetDate',
    'y_bin',
    'y_num'
    ], axis=1)
y = df['y_bin']

In [8]:
y

0     0
1     1
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    1
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    1
36    0
37    0
38    0
39    0
40    0
41    0
42    1
43    0
44    0
45    0
46    0
47    0
48    0
Name: y_bin, dtype: int64

In [9]:
n_experiment = 10
ex_name = '0514'
data = 'id1504'

In [12]:
print('MTS開始！')


result_df = pd.DataFrame(
                    columns=['AUC', 'accuracy', 'recall', 'Specificity', 'precision', 'gmeans', 'RS'],
                    index=range(n_experiment))
for m in tqdm(range(n_experiment)):
    # バギング側の話
    # ブートストラップサンプリングの個数
    n = 10
    seed = random.randint(0, n)

    # 使用する7つの変数をランダムに取得する
    # バギングをする際はそれぞれのサブサンプルで7つの変数を選択する．
    random.seed(seed)
    random_s = random.sample(list(X.columns), len(X.columns) if len(X.columns) < 7 else 7)
    X = X[random_s]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

    result_scaler, result_inv_C, select_columns = fit_MTS(X_train, y_train)

    y_train_pred = predict_MD(X_train, result_scaler, result_inv_C, select_columns)

    threshold = determine_threshold(y_train, y_train_pred)

    y_proba = predict_MD(X_test, result_scaler, result_inv_C, select_columns)
    y_pred = y_proba > threshold

    result_df = make_result_df(result_df, y_test, y_pred, y_proba, m)
    
result_df.to_csv(f'../data/output/{ex_name}_MTS_{data}_result.csv')

MTS開始！


  0%|          | 0/10 [00:00<?, ?it/s]c:\Users\baseb\Documents\github\修論\SMOTEMTSBag\MTSBag\code\func\my_func.py:574: RuntimeWarning: divide by zero encountered in double_scalars
  result_df['RS'][m] = (TP / (TP + FN)) / (TN / (TN + FP))
c:\Users\baseb\Documents\github\修論\SMOTEMTSBag\MTSBag\code\func\my_func.py:574: RuntimeWarning: divide by zero encountered in double_scalars
  result_df['RS'][m] = (TP / (TP + FN)) / (TN / (TN + FP))
 30%|███       | 3/10 [00:00<00:00, 25.85it/s]c:\Users\baseb\Documents\github\修論\SMOTEMTSBag\MTSBag\code\func\my_func.py:574: RuntimeWarning: divide by zero encountered in double_scalars
  result_df['RS'][m] = (TP / (TP + FN)) / (TN / (TN + FP))
c:\Users\baseb\Documents\github\修論\SMOTEMTSBag\MTSBag\code\func\my_func.py:571: RuntimeWarning: invalid value encountered in longlong_scalars
  result_df['precision'][m] = TP / (TP + FP)
c:\Users\baseb\Documents\github\修論\SMOTEMTSBag\MTSBag\code\func\my_func.py:574: RuntimeWarning: divide by zero encountered in dou